In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
data=pd.read_csv('final_data.csv')

In [3]:
data1=data.copy()

In [4]:
data1.drop(['crop_type'],axis=1,inplace=True)

In [5]:
data1.head()

,pH,N,P,K,OC,Particles,Water_holding_content,Soil_type
0,7.03,118.72,25.00,185.82,0.78,0.001686,1.40,clay
1,2.57,131.09,24.62,181.71,0.90,0.000381,1.31,clay
2,3.39,104.32,18.37,202.39,0.76,0.000226,1.35,clay
3,2.73,111.21,21.09,202.71,0.68,0.001399,1.22,clay
4,1.52,90.90,23.76,195.39,0.74,0.001587,1.21,clay


In [6]:
data.head()

,pH,N,P,K,OC,Particles,Water_holding_content,Soil_type,crop_type
0,7.03,118.72,25.00,185.82,0.78,0.001686,1.40,clay,carrot
1,2.57,131.09,24.62,181.71,0.90,0.000381,1.31,clay,carrot
2,3.39,104.32,18.37,202.39,0.76,0.000226,1.35,clay,carrot
3,2.73,111.21,21.09,202.71,0.68,0.001399,1.22,clay,carrot
4,1.52,90.90,23.76,195.39,0.74,0.001587,1.21,clay,carrot


In [42]:
data.corr()

,pH,N,P,K,OC,Particles,Water_holding_content
pH,1.000000,-0.712106,-0.724899,-0.715799,-0.679279,0.334492,0.418809
N,-0.712106,1.000000,0.818275,0.845193,0.845460,-0.578084,-0.175351
P,-0.724899,0.818275,1.000000,0.822211,0.802546,-0.477921,-0.295212
K,-0.715799,0.845193,0.822211,1.000000,0.861902,-0.598714,-0.141853
OC,-0.679279,0.845460,0.802546,0.861902,1.000000,-0.666949,-0.026386
Particles,0.334492,-0.578084,-0.477921,-0.598714,-0.666949,1.000000,-0.413970
Water_holding_content,0.418809,-0.175351,-0.295212,-0.141853,-0.026386,-0.413970,1.000000


In [7]:
len(data['crop_type'].unique())

10

In [129]:
data1.isnull().any()

pH                       False
N                        False
P                        False
K                        False
OC                       False
Particles                False
Water_holding_content    False
dtype: bool

In [9]:
label=data['crop_type']

In [26]:
data1.drop('Soil_type',axis=1,inplace=True)

In [27]:
y=pd.get_dummies(label)

In [95]:
Y=label

In [96]:
labels=y.columns

In [28]:
X=pd.get_dummies(data1)


In [12]:
X.shape

(999, 10)

In [13]:
from sklearn.model_selection import train_test_split

In [103]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=0,test_size=.15)

In [138]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

In [130]:
gd=GridSearchCV(RandomForestClassifier(),{'n_estimators':[50,100,150],'max_depth':[3,5,9]},cv=5).fit(X,Y)

In [144]:
kd=GridSearchCV(KNeighborsClassifier(),{'n_neighbors':[5,7,9,11]},cv=5).fit(X,Y)

In [145]:
gd.best_params_

{'max_depth': 3, 'n_estimators': 50}

In [146]:
kd.best_params_

{'n_neighbors': 9}

In [147]:
gd.best_estimator_.score(X,Y)

0.46046046046046046

In [148]:
kd.best_estimator_.score(X,Y)

0.5385385385385385

In [149]:
kd.best_estimator_.score(X_train,y_train)

0.5265017667844523

In [150]:
kd.best_estimator_.score(X_test,y_test)

0.6066666666666667

In [73]:
import tensorflow as tf

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization
from tensorflow.keras.models import load_model 
from kerastuner.tuners import RandomSearch 
from tensorflow.keras.optimizers import Adam

In [78]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(units=hp.Int('layer1',min_value=30,max_value=240,step=30),input_dim=7,activation='relu'))
    model.add(Dense(units=hp.Int('layer2',min_value=30,max_value=240,step=30),activation='relu'))
    model.add(Dense(units=hp.Int('layer3',min_value=30,max_value=240,step=30),activation='relu'))
    model.add(BatchNormalization())    
    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=Adam(hp.Choice('lr',values=[0.1,0.01,0.001])),metrics=['accuracy'])
    return model

In [79]:
tuner=RandomSearch(build_model,objective='val_accuracy',max_trials=5,project_name='trail')

In [80]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Train on 849 samples, validate on 150 samples
Epoch 1/5
849/849 [==============================] - ETA: 2:55 - loss: 2.9040 - accuracy: 0.0000e+ - ETA: 39s - loss: 2.0007 - accuracy: 0.2969     - ETA: 30s - loss: 1.9777 - accuracy: 0.281 - ETA: 13s - loss: 1.8661 - accuracy: 0.274 - ETA: 7s - loss: 1.7743 - accuracy: 0.293 - ETA: 2s - loss: 1.6949 - accuracy: 0.32 - ETA: 0s - loss: 1.6446 - accuracy: 0.32 - 9s 10ms/sample - loss: 1.6413 - accuracy: 0.3251 - val_loss: 23.5188 - val_accuracy: 0.1667
Epoch 2/5
849/849 [==============================] - ETA: 0s - loss: 1.4414 - accuracy: 0.43 - ETA: 0s - loss: 1.4369 - accuracy: 0.43 - ETA: 0s - loss: 1.4634 - accuracy: 0.41 - ETA: 0s - loss: 1.4607 - accuracy: 0.40 - ETA: 0s - loss: 1.4807 - accuracy: 0.37 - ETA: 0s - loss: 1.4788 - accuracy: 0.37 - ETA: 0s - loss: 1.4952 - accuracy: 0.37 - 1s 879us/sample - loss: 1.4826 - accuracy: 0.3828 - val_loss: 12.2405 - val_accuracy: 0.1667
Epoch 3/5
849/849 [==============================] - ETA:

Train on 849 samples, validate on 150 samples
Epoch 1/5
849/849 [==============================] - ETA: 46s - loss: 2.1606 - accuracy: 0.187 - ETA: 1s - loss: 2.1435 - accuracy: 0.290 - 3s 3ms/sample - loss: 1.9182 - accuracy: 0.2980 - val_loss: 42.8538 - val_accuracy: 0.1667
Epoch 2/5
849/849 [==============================] - ETA: 0s - loss: 1.6178 - accuracy: 0.21 - ETA: 0s - loss: 1.4942 - accuracy: 0.32 - 0s 184us/sample - loss: 1.5003 - accuracy: 0.3333 - val_loss: 18.3670 - val_accuracy: 0.1667
Epoch 3/5
849/849 [==============================] - ETA: 0s - loss: 1.5911 - accuracy: 0.34 - ETA: 0s - loss: 1.5565 - accuracy: 0.32 - 0s 166us/sample - loss: 1.5542 - accuracy: 0.3404 - val_loss: 3.1564 - val_accuracy: 0.1467
Epoch 4/5
849/849 [==============================] - ETA: 0s - loss: 1.2633 - accuracy: 0.46 - ETA: 0s - loss: 1.6748 - accuracy: 0.29 - ETA: 0s - loss: 1.6640 - accuracy: 0.28 - 0s 221us/sample - loss: 1.6333 - accuracy: 0.3004 - val_loss: 2.7960 - val_accuracy: 

Train on 849 samples, validate on 150 samples
Epoch 1/5
849/849 [==============================] - ETA: 42s - loss: 2.8878 - accuracy: 0.0000e+0 - ETA: 1s - loss: 1.8636 - accuracy: 0.2837    - ETA: 0s - loss: 1.7385 - accuracy: 0.29 - 3s 3ms/sample - loss: 1.7027 - accuracy: 0.3062 - val_loss: 21.0761 - val_accuracy: 0.1333
Epoch 2/5
849/849 [==============================] - ETA: 0s - loss: 1.5981 - accuracy: 0.34 - ETA: 0s - loss: 1.4173 - accuracy: 0.42 - 0s 166us/sample - loss: 1.4111 - accuracy: 0.3993 - val_loss: 15.4450 - val_accuracy: 0.0800
Epoch 3/5
849/849 [==============================] - ETA: 0s - loss: 1.4459 - accuracy: 0.31 - ETA: 0s - loss: 1.4872 - accuracy: 0.28 - ETA: 0s - loss: 1.3980 - accuracy: 0.36 - 1s 1ms/sample - loss: 1.4023 - accuracy: 0.3769 - val_loss: 13.5287 - val_accuracy: 0.2067
Epoch 4/5
849/849 [==============================] - ETA: 0s - loss: 1.2236 - accuracy: 0.46 - ETA: 0s - loss: 1.3313 - accuracy: 0.35 - ETA: 0s - loss: 1.3685 - accuracy: 0

Train on 849 samples, validate on 150 samples
Epoch 1/5
849/849 [==============================] - ETA: 36s - loss: 2.8861 - accuracy: 0.125 - ETA: 1s - loss: 2.2152 - accuracy: 0.209 - ETA: 0s - loss: 2.0146 - accuracy: 0.25 - 2s 3ms/sample - loss: 2.0080 - accuracy: 0.2580 - val_loss: 5.3604 - val_accuracy: 0.1467
Epoch 2/5
849/849 [==============================] - ETA: 0s - loss: 1.7156 - accuracy: 0.31 - ETA: 0s - loss: 1.6688 - accuracy: 0.33 - ETA: 0s - loss: 1.6488 - accuracy: 0.33 - ETA: 0s - loss: 1.6182 - accuracy: 0.35 - 0s 274us/sample - loss: 1.6136 - accuracy: 0.3522 - val_loss: 3.8746 - val_accuracy: 0.1133
Epoch 3/5
849/849 [==============================] - ETA: 0s - loss: 1.4407 - accuracy: 0.46 - ETA: 0s - loss: 1.5122 - accuracy: 0.37 - ETA: 0s - loss: 1.5202 - accuracy: 0.37 - 1s 782us/sample - loss: 1.5171 - accuracy: 0.3746 - val_loss: 3.1539 - val_accuracy: 0.1667
Epoch 4/5
849/849 [==============================] - ETA: 0s - loss: 1.3929 - accuracy: 0.53 - ETA

Train on 849 samples, validate on 150 samples
Epoch 1/5
849/849 [==============================] - ETA: 41s - loss: 3.4204 - accuracy: 0.062 - ETA: 1s - loss: 1.9364 - accuracy: 0.266 - ETA: 0s - loss: 1.7525 - accuracy: 0.31 - 3s 3ms/sample - loss: 1.7446 - accuracy: 0.3192 - val_loss: 24.5826 - val_accuracy: 0.1667
Epoch 2/5
849/849 [==============================] - ETA: 0s - loss: 1.4575 - accuracy: 0.37 - ETA: 0s - loss: 1.4484 - accuracy: 0.41 - ETA: 0s - loss: 1.4612 - accuracy: 0.40 - 0s 173us/sample - loss: 1.4524 - accuracy: 0.3993 - val_loss: 12.7409 - val_accuracy: 0.0800
Epoch 3/5
849/849 [==============================] - ETA: 0s - loss: 1.4420 - accuracy: 0.31 - ETA: 0s - loss: 1.3935 - accuracy: 0.36 - ETA: 0s - loss: 1.3755 - accuracy: 0.36 - 0s 236us/sample - loss: 1.4015 - accuracy: 0.3734 - val_loss: 5.5194 - val_accuracy: 0.1667
Epoch 4/5
849/849 [==============================] - ETA: 0s - loss: 1.6817 - accuracy: 0.18 - ETA: 0s - loss: 1.4040 - accuracy: 0.35 - E

INFO:tensorflow:Oracle triggered exit


In [81]:
model=tuner.get_best_models(num_models=1)[0]

In [82]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 120)               960       
_________________________________________________________________
dense_1 (Dense)              (None, 240)               29040     
_________________________________________________________________
dense_2 (Dense)              (None, 180)               43380     
_________________________________________________________________
batch_normalization (BatchNo (None, 180)               720       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1810      
Total params: 75,910
Trainable params: 75,550
Non-trainable params: 360
_________________________________________________________________


In [84]:
model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test))

Train on 849 samples, validate on 150 samples
Epoch 1/50
849/849 [==============================] - ETA: 1s - loss: 1.4155 - accuracy: 0.37 - ETA: 0s - loss: 1.3587 - accuracy: 0.36 - ETA: 0s - loss: 1.3551 - accuracy: 0.36 - ETA: 0s - loss: 1.3170 - accuracy: 0.37 - ETA: 0s - loss: 1.3004 - accuracy: 0.41 - ETA: 0s - loss: 1.2887 - accuracy: 0.40 - ETA: 0s - loss: 1.2885 - accuracy: 0.38 - 0s 533us/sample - loss: 1.2854 - accuracy: 0.3899 - val_loss: 4.8794 - val_accuracy: 0.1467
Epoch 2/50
849/849 [==============================] - ETA: 0s - loss: 1.2020 - accuracy: 0.46 - ETA: 0s - loss: 1.2560 - accuracy: 0.40 - ETA: 0s - loss: 1.2697 - accuracy: 0.37 - ETA: 0s - loss: 1.2868 - accuracy: 0.37 - 0s 298us/sample - loss: 1.2975 - accuracy: 0.3675 - val_loss: 9.0336 - val_accuracy: 0.0467
Epoch 3/50
849/849 [==============================] - ETA: 0s - loss: 1.2325 - accuracy: 0.46 - ETA: 0s - loss: 1.2829 - accuracy: 0.43 - ETA: 0s - loss: 1.2525 - accuracy: 0.42 - ETA: 0s - loss: 1.25

849/849 [==============================] - ETA: 0s - loss: 1.0754 - accuracy: 0.59 - ETA: 0s - loss: 1.1681 - accuracy: 0.42 - ETA: 0s - loss: 1.1826 - accuracy: 0.39 - 0s 221us/sample - loss: 1.1795 - accuracy: 0.3981 - val_loss: 1.2206 - val_accuracy: 0.4667
Epoch 27/50
849/849 [==============================] - ETA: 0s - loss: 1.1976 - accuracy: 0.40 - ETA: 0s - loss: 1.1617 - accuracy: 0.43 - ETA: 0s - loss: 1.1915 - accuracy: 0.41 - 0s 174us/sample - loss: 1.1952 - accuracy: 0.4099 - val_loss: 1.9209 - val_accuracy: 0.2600
Epoch 28/50
849/849 [==============================] - ETA: 0s - loss: 1.2108 - accuracy: 0.43 - ETA: 0s - loss: 1.1919 - accuracy: 0.44 - ETA: 0s - loss: 1.2352 - accuracy: 0.41 - ETA: 0s - loss: 1.2032 - accuracy: 0.41 - 0s 302us/sample - loss: 1.1929 - accuracy: 0.4217 - val_loss: 1.2775 - val_accuracy: 0.4133
Epoch 29/50
849/849 [==============================] - ETA: 0s - loss: 1.1785 - accuracy: 0.40 - ETA: 0s - loss: 1.1641 - accuracy: 0.42 - ETA: 0s - lo

In [91]:
model.evaluate(X_train,y_train)

849/849 [==============================] - ETA: 0s - loss: 1.3889 - accuracy: 0.40 - ETA: 0s - loss: 1.3105 - accuracy: 0.38 - 0s 91us/sample - loss: 1.3028 - accuracy: 0.3734


[1.3028306167735086, 0.37338045]

In [92]:
model.save('crop recommendation model 2.h5')

In [114]:
a=np.array([
7.03,
118.72,
25.00,
185.82,
0.78,
0.001686,
1.40])

In [125]:
a=a.reshape(1,7)

In [128]:
labels[np.argmax(model.predict(a),axis=1)][0]

'carrot'

In [151]:
kd.best_estimator_.predict(a)

array(['rice'], dtype=object)